In [ ]:
import os
os.environ['VLLM_USE_V1'] = '0'
from indextts.infer_v2_vllm import IndexTTS2
# from indextts.infer_v2 import IndexTTS2
import asyncio
index_tts = IndexTTS2(cfg_path="checkpoints/config.yaml", model_dir="checkpoints", use_cuda_kernel=False, gpu_memory_utilization=0.25)

In [ ]:
import importlib
import types
from pydub import AudioSegment
from indextts import infer_v2_vllm
importlib.reload(infer_v2_vllm)
from indextts.infer_v2_vllm import IndexTTS2
index_tts.infer_stream = types.MethodType(IndexTTS2.infer_stream, index_tts)
index_tts.infer_parallel = types.MethodType(IndexTTS2.infer_parallel, index_tts)

count = 0
concat_segment = None
from io import BytesIO

async for chunk in index_tts.infer_stream(spk_audio_prompt='data/tina_test.wav', text="""Tom could benefit from more consistent practice to really solidify his understanding and skills in chess. This is a common area for growth, and with some focused effort, we'll see great progress. To help Tom get more accustomed to chess, we recommend establishing a fun, regular practice routine at home. Tom could try playing one or two quick games daily, either with family or on a platform like ChessKid. Solving a few chess puzzles each week through an app or workbook is also an excellent way to sharpen his mind. Aiming for around 2-4 hours of practice a week will build a strong foundation and keep the game enjoyable and engaging for him.""", output_path="test_parallel2.wav", verbose=False, max_text_tokens_per_segment=120, interval_silence=0):


    with open(f"test_stream_{count}.wav", "wb") as f:
        f.write(chunk)
    if concat_segment is None:
        concat_segment = AudioSegment.from_file(BytesIO(chunk))
    else:
        concat_segment = concat_segment.append(AudioSegment.from_file(BytesIO(chunk)), crossfade=0)

concat_segment.export("test_stream.mp3", format="mp3")
# await index_tts.infer_parallel(spk_audio_prompt='data/test_real.wav', text="""Tom could benefit from more consistent practice to really solidify his understanding and skills in chess. This is a common area for growth, and with some focused effort, we'll see great progress. To help Tom get more accustomed to chess, we recommend establishing a fun, regular practice routine at home. Tom could try playing one or two quick games daily, either with family or on a platform like ChessKid. Solving a few chess puzzles each week through an app or workbook is also an excellent way to sharpen his mind. Aiming for around 2-4 hours of practice a week will build a strong foundation and keep the game enjoyable and engaging for him.""", output_path="test_parallel.wav", verbose=False, max_text_tokens_per_segment=120, interval_silence=0)


In [ ]:
from pydub import AudioSegment

# 加载两个音频文件
audio1 = AudioSegment.from_mp3("test_stream_1.wav")
audio2 = AudioSegment.from_mp3("test_stream_2.wav")

merged_chunk = audio1.append(audio2, crossfade=0)
merged_chunk.export("merged_chunk.mp3", format="mp3")
# def stream_with_crossfade(audio1, audio2, crossfade_ms=250):
#     # Step 1: 播放 audio1 主体部分（除去最后 crossfade_ms）
#     main_part = audio1[:-crossfade_ms]
#     yield main_part.raw_data  # 或导出为临时文件/分块传输

#     # Step 2: 交叉淡出淡入部分
#     fade_out = audio1[-crossfade_ms:].fade_out(crossfade_ms)
#     fade_in = audio2[:crossfade_ms].fade_in(crossfade_ms)
#     crossfaded = fade_out.overlay(fade_in)  # 混合
#     yield crossfaded.raw_data

#     # Step 3: 播放 audio2 剩余部分
#     remaining = audio2[crossfade_ms:]
#     yield remaining.raw_data
# crossfade_ms = 250
# a1 = audio1[:-crossfade_ms]
# a2 = audio2[crossfade_ms:]
# fade_out = audio1[-crossfade_ms:].fade_out(crossfade_ms)
# fade_in = audio2[:crossfade_ms].fade_in(crossfade_ms)
# crossfaded = fade_out.overlay(fade_in)

# merged_chunk = a1.append(crossfaded, crossfade=0)
# merged_chunk = merged_chunk.append(a2, crossfade=0)
# merged_chunk.export("merged_chunk.mp3", format="mp3")
# with open("merged_chunk.mp3", "wb") as f:
    # f.write(a1 + crossfaded + a2)


In [27]:
import httpx
import requests
from pydub import AudioSegment
from io import BytesIO
import time
payload = {
    "prompt_audio_url": "https://beezstorageblob.blob.core.windows.net/emailblob/cloned_voice_0e3a2d2d-7a24-41d2-b1f5-c7af7f3ab284.wav",
    "text": "Tom could benefit from more consistent practice to really solidify his understanding and skills in chess. This is a common area for growth, and with some focused effort, we'll see great progress. To help Tom get more accustomed to chess, we recommend establishing a fun, regular practice routine at home. Tom could try playing one or two quick games daily, either with family or on a platform like ChessKid. Solving a few chess puzzles each week through an app or workbook is also an excellent way to sharpen his mind. Aiming for around 2-4 hours of practice a week will build a strong foundation and keep the game enjoyable and engaging for him.",
}

concat_segment = None




try:
    with httpx.stream("POST", "http://127.0.0.1:62343/generate-speech-stream/", json=payload, follow_redirects=True) as r:
        # r.raise_for_status() # Ensure the request was successful
        start_time = time.time()
        for chunk in r.iter_bytes():
            end_time = time.time()
            print(f"Time taken to receive chunk: {end_time - start_time:.2f} seconds")
            start_time = end_time
            if concat_segment is None:
                concat_segment = AudioSegment.from_file(BytesIO(chunk))
            else:
                concat_segment = concat_segment.append(AudioSegment.from_file(BytesIO(chunk)), crossfade=0)
    concat_segment.export("test_stream.mp3", format="mp3")
except httpx.HTTPError as e:
    print(f"An HTTP error occurred: {e}")

Time taken to receive chunk: 1.27 seconds
Time taken to receive chunk: 1.13 seconds
Time taken to receive chunk: 1.32 seconds
Time taken to receive chunk: 1.17 seconds
Time taken to receive chunk: 1.25 seconds
Time taken to receive chunk: 1.31 seconds


In [28]:
# transcribe base64
import base64
import requests
# with open("test_stream.mp3", "rb") as f:
with open("test_stream.mp3", "rb") as f:
    data = f.read()
    base64_data = base64.b64encode(data).decode("utf-8")
payload = {
    "audio_data": base64_data,
    "file_extension": "wav",
    "language": "en"
}

response = requests.post("http://127.0.0.1:62343/transcribe_base64/", json=payload)
print(response.json())

{'text': "Tom could benefit from more consistent practice to really solidify his understanding and skills in chess. This is a common area for growth and with some focused effort, we'll see great progress. To help Tom get more accustomed to chess, we recommend establishing a fun, regular practice routine at home. Tom could try playing one or two quick games daily, either with family or on a platform like ChessKit. Solving a few chess puzzles each week through an app or workbook is also an excellent way to sharpen his mind. Aiming for around 24 hours of practice a week will build a strong foundation and keep the game enjoyable and engaging for him.", 'segments': [{'start': 0.0, 'end': 29.44, 'text': "Tom could benefit from more consistent practice to really solidify his understanding and skills in chess. This is a common area for growth and with some focused effort, we'll see great progress. To help Tom get more accustomed to chess, we recommend establishing a fun, regular practice routi